<a href="https://colab.research.google.com/github/xxxttttz/Transformer_Learn/blob/main/TransFormer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import math

class selfAttentionV4(nn.Module):
  def __init__(self, hidden_dim,dropout_rate=0.1) -> None:
     super().__init__()
     self.hidden_dim = hidden_dim

     self.Q = nn.Linear(hidden_dim,hidden_dim)
     self.K = nn.Linear(hidden_dim,hidden_dim)
     self.V = nn.Linear(hidden_dim,hidden_dim)

     self.dropout_rate = nn.Dropout(dropout_rate)

  def forward(self,X,attention_mask=None):
    #X shape(bitch seq hidden_dim)
    Q = self.Q(X)
    K = self.K(X)
    V = self.V(X)
    #shape bit seq seq
    attention_weight = Q @ K.transpose(-2,-1) / math.sqrt(self.hidden_dim)

    # mask shape (bit seq seq)
    if attention_mask is not None:
      attention_weight = attention_weight.masked_fill(
          attention_mask == 0,
          float("-inf")
      )
    attention_weight = torch.softmax(attention_weight,-1)
    print(attention_weight)
    attention_weight = self.dropout_rate(attention_weight)
    #bit seq hidden_idm
    output = attention_weight @ V
    return output

#bit seq hidden_dim
X = torch.rand(3,4,2)
# shape (3,4)
mask = torch.tensor([
    [1,1,1,0],
    [1,1,0,0],
    [1,0,0,0]
])
#shape 3 4 4
mask = mask.unsqueeze(1).repeat(1,4,1)

net = selfAttentionV4(2)
net(X,mask)




tensor([[[0.3231, 0.3264, 0.3505, 0.0000],
         [0.3196, 0.3230, 0.3573, 0.0000],
         [0.3132, 0.3232, 0.3636, 0.0000],
         [0.3186, 0.3247, 0.3567, 0.0000]],

        [[0.4731, 0.5269, 0.0000, 0.0000],
         [0.4686, 0.5314, 0.0000, 0.0000],
         [0.4759, 0.5241, 0.0000, 0.0000],
         [0.4757, 0.5243, 0.0000, 0.0000]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000]]], grad_fn=<SoftmaxBackward0>)


tensor([[[-0.0655,  0.0211],
         [-0.0649,  0.0243],
         [-0.0638,  0.0277],
         [-0.0647,  0.0242]],

        [[-0.0447,  0.0773],
         [-0.0125, -0.0775],
         [-0.0446,  0.0761],
         [-0.0447,  0.0761]],

        [[-0.0485, -0.3144],
         [-0.0485, -0.3144],
         [-0.0485, -0.3144],
         [ 0.0000,  0.0000]]], grad_fn=<UnsafeViewBackward0>)

In [1]:
import torch
import torch.nn as nn
import math


class MultiHeadAttention(nn.Module):
  def __init__(self,hidden_dim,head_num):
     super().__init__()

     self.hidden_dim = hidden_dim
     self.head_num = head_num
     self.head_dim = hidden_dim // head_num

     self.query_proj = nn.Linear(hidden_dim,hidden_dim)
     self.key_proj = nn.Linear(hidden_dim,hidden_dim)
     self.value_proj = nn.Linear(hidden_dim,hidden_dim)

     self.att_dropout = nn.Dropout(0.1)

     self.output_proj = nn.Linear(hidden_dim,hidden_dim)

  def forward(self,X,attention_mask = None):
    bitch_size,seq_len,_ = X.shape

    Q = self.query_proj(X)
    K = self.key_proj(X)
    V = self.value_proj(X)

    #shape (bit seq hidden_dim)=> (bit head_num seq head_dim)
    Q = Q.view(bitch_size,seq_len,self.head_num,self.head_dim).transpose(1,2)
    K = K.view(bitch_size,seq_len,self.head_num,self.head_dim).permute(
        0,2,1,3
    )
    V = V.view(bitch_size,seq_len,self.head_num,self.head_dim).transpose(1,2)

    #shape (bit head_num seq seq)

    attention_weight = Q @ K.transpose(-1,-2)/math.sqrt(self.head_dim)
    if attention_mask is not None:
      attention_weight = attention_weight.masked_fill(
          attention_mask==0,
          float("-1e20")
      )


    #softmax
    attention_weight = torch.softmax(attention_weight,-1)
    print(attention_weight)

    attention_weight = self.att_dropout(attention_weight)

    #shape bit head_num seq head_dim
    print(attention_weight.shape)
    print(V.shape)
    out_mid = attention_weight @ V

    out_mid = out_mid.transpose(1,2).contiguous()
    output = out_mid.view(bitch_size,seq_len,-1)
    print(f"outputsize is {output.shape}")#[3, 2, 128]
    output = self.output_proj(output)
    print(f"output is {output}")
    return output

attention_mask = (torch.tensor(
        [
            [0, 1],
            [0, 0],
            [1, 0],
        ]
    )
    .unsqueeze(1)
    .unsqueeze(2)
    .expand(3, 8, 2, 2)
)
print(f"attention_mask shape is {attention_mask.shape}")
x = torch.rand(3, 2, 128)
net = MultiHeadAttention(128, 8)
net(x, attention_mask).shape






attention_mask shape is torch.Size([3, 8, 2, 2])
tensor([[[[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]],

         [[0.0000, 1.0000],
          [0.0000, 1.0000]]],


        [[[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]],

         [[0.5000, 0.5000],
          [0.5000, 0.5000]]],


        [[[1.0000, 

torch.Size([3, 2, 128])

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git clone <https://github.com/xxxttttz/Transformer_Learn.git>'
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `git add <TransFormer_learning.ipynb>'
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
